In [1]:
import math
import numpy as np
import math
# import pandas
from optparse import OptionParser
from sklearn.tree import DecisionTreeRegressor
from sklearn import ensemble
from collections import defaultdict
from copy import deepcopy
from multiprocessing import Pool
from itertools import chain
import time
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import os

In [2]:
# Functions to extrac the documents, query and rank information
def extractFeatures(split):
    features = []
    for i in range(2, 138):
        features.append(float(split[i].split(':')[1]))
    # Convert to tuples:
    return features

def extractQueryData(split):
    # Add tuples:
    queryFeatures = split[1].split(':')[1]
    return queryFeatures

def readDataset(path):
    dictio_quid= defaultdict(list)
    features_list=[]
    rank_list=[]
    print('Reading training data from file...')
    k=0
    with open(path, 'r') as file:
        for line in file:
            # Getting features:
            split = line.split()
            features=extractFeatures(split)
            features_list.append(features)

            # Getting the query:
            query = int(extractQueryData(split))
            #print "query: "+str(query)
            # Getting rank:
            rank = int(split[0])
            rank_list.append(rank)

            # Feeding dictionary:
            dictio_quid[query].append((features, rank))
            k+=1
            #if k==10:
            #   break
    print('Number of query ID %d' %(len(features_list)))
    return np.array(features_list), np.array(rank_list), dictio_quid

# Normalisation:
def normalize_features(features):
    features=np.array(features)

    # Substracting the mean:
    mean_features = np.mean(features, axis=0)
    features = features - mean_features

    # Dividing by the std:
    std_features = np.std(features, axis=0)
    features = features / std_features
    #print "features normalized"
    return features, mean_features, std_features

In [3]:
features, ranks, queries= readDataset('LEMUR/MSLR-WEB10K/Fold1/train.txt')
features, _, _= normalize_features(features)

Reading training data from file...
Number of query ID 723412


In [4]:
tes_features, tes_ranks, tes_queries= readDataset('LEMUR/MSLR-WEB10K/Fold1/test.txt')
tes_features, mean_Xval, std_Xval= normalize_features(tes_features)

Reading training data from file...
Number of query ID 241521


In [5]:
ranks_r = np.asarray(ranks).reshape(-1, 1)
enc = OneHotEncoder()
enc.fit(ranks_r)
ranks_enc = enc.transform(ranks_r).toarray()
queries_r = np.asarray(queries).reshape(-1, 1)

In [6]:
tes_ranks_r = np.asarray(tes_ranks).reshape(-1, 1)
tes_ranks_enc = enc.transform(tes_ranks_r).toarray()
tes_queries_r = np.asarray(tes_queries).reshape(-1, 1)

In [7]:
def reorder_dictio(dictio_eval):
    for key in dictio_eval.keys():
        #dictio_eval[key]=sorted(dictio_eval[key], reverse=True,key=lambda tup: (tup[1], tup[0]))
        dictio_eval[key] = sorted(dictio_eval[key], reverse=True, key=lambda tup: tup[1])
    return dictio_eval

def dcg_score(y_true, y_score, k=10, gains="exponential"):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")

    # highest rank is 1 so +2 instead of +1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

def ndcg_score(y_true, y_score, k=10, gains="exponential"):
    best = dcg_score(y_true, y_true, k, gains)
    actual = dcg_score(y_true, y_score, k, gains)
    return actual / best

def separate(relevance_tuple):
    y_true=[]
    y_pred = []
    for tup in relevance_tuple:
        y_pred.append(tup[0])
        y_true.append(tup[1])
    return y_true,y_pred

def ndcg(dictio_eval):
    """
    """
    b= reorder_dictio(dictio_eval)
    new_b=[]
    for qid in b.keys():
        r_true=[]
        r_pred=[]
        for i,j in zip(pd.DataFrame(b[qid])[0], pd.DataFrame(b[qid])[1]):
            r_pred.append(i)
            r_true.append(j)
        score=ndcg_score(r_true,r_pred)    
        #new_b[qid]= [r_true, r_pred]
        new_b.append(score)
    return new_b

In [ ]:
def get_bins(dictio_eval):
    #Get the bins:
    all_tuples=[]
    for key in dictio_eval.keys():
        all_tuples.append(dictio_eval[key])
    all_tuples=list(chain.from_iterable(all_tuples))
    all_scores=[i[0] for i in all_tuples]
    all_true=[i[1] for i in all_tuples]
    number_bins=4#len(set(all_true))
    bins = np.histogram(all_scores, bins=number_bins, range=None, normed=False, weights=None)[1]
    return bins

def assign_to_bin(list_score,bins):
    # Get bins
    #bins = np.histogram(list_score, bins=number_bins, range=None, normed=False, weights=None)[1]
    inds = np.digitize(list_score, bins,right=True)
    return inds

# Functions to calculate ERR:
GAMMA=0.450

# Functions to calculate ERR:
def get_proba(list_tuples):
    list_proba=[]
    for r_pred,r_true in list_tuples:
        proba = (np.power(2,r_pred)-1)/np.power(2,4)
        list_proba.append(proba)
    return list_proba

def get_ERR(list_proba,n=10,gamma=0.5):
    r=2
    err = list_proba[0]
    last_proba=1
    for i in xrange(1,len(list_proba)):
        actual_proba=list_proba[i]
        previous_proba=(1-list_proba[i-1])*last_proba
        #print proba
        stop_proba=actual_proba*previous_proba
        err+=stop_proba/r
        last_proba=previous_proba
        r+=1
    return err

def ERR(dictio_eval,n=10,gamma=0.5):
    list_ERR=[]
    for key in dictio_eval.keys():
        list_tuples=dictio_eval[key]
        list_proba=get_proba(list_tuples)
        err_result=get_ERR(list_proba,n,gamma)
        list_ERR.append(err_result)
    return list_ERR

In [9]:
def evaluate(model, dictio_val, mean_Xval, std_Xval):
    dictio_evaluation = defaultdict(list)

    for key in dictio_val.keys():
        temp_list = dictio_val[key]
        for features_vec, relevance in temp_list:
            # Features:
            features_norm = (np.array(features_vec) - mean_Xval) / std_Xval
            features_norm = features_norm.reshape(1,-1)
            #features_norm = features_norm.reshape(-1,1)

            # Prediction:
            prediction = model.predict(features_norm)

            # Dictionary:
            dictio_evaluation[key].append((prediction[0], relevance))

            # print features_vec,relevance

    return dictio_evaluation

In [18]:
class TF_LogReg():
    
    def __init__(self):
        self.self = self
        
    def initilise_model(self):
        
        tf.reset_default_graph()
        
        self.x_i = tf.placeholder("float", [None, 136])
        self.y_i = tf.placeholder("float", [None, 5])
        self.qid = tf.placeholder("float", [None, 1])

        d_in = 136
        d_out = 5

        self.W1 = tf.Variable(tf.random_normal([d_in, d_out], mean= 0.01, stddev= 0.01))
        self.b1 = tf.Variable(tf.random_normal([d_out], mean= 0.01, stddev= 0.01))

        self.a1 = tf.matmul(self.x_i, self.W1)+ self.b1
        self.y_hat = self.a1

        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= self.y_i, logits=self.y_hat))
        
        self.global_step = tf.Variable(0, trainable=False)
        self.starter_learning_rate = 0.0001
        self.learning_rate = tf.train.exponential_decay(self.starter_learning_rate, self.global_step,
                                                   100000, 0.96, staircase=True)
        self.optimiser = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss) 
        
        self.prediction = tf.argmax(self.y_hat, axis=1)
        self.correct = tf.argmax(self.y_i, axis=1)
        self.mistakes = tf.not_equal(self.correct, self.prediction)
        self.accuracy = 1- tf.reduce_mean(tf.cast(self.mistakes, tf.float32))
        
        self.saver = tf.train.Saver()

    def initilise_model_linear(self):
        
        tf.reset_default_graph()
        
        self.x_i = tf.placeholder("float", [None, 136])
        self.y_i = tf.placeholder("float", [None, 1])
        self.qid = tf.placeholder("float", [None, 1])

        d_in = 136
        d_out = 1

        self.W1 = tf.Variable(tf.random_normal([d_in, d_out], mean= 0.01, stddev= 0.01))
        self.b1 = tf.Variable(tf.random_normal([d_out], mean= 0.01, stddev= 0.01))

        self.a1 = tf.matmul(self.x_i, self.W1)+ self.b1
        self.y_hat = tf.nn.sigmoid(self.a1)* 4

        self.correct = tf.argmax(self.y_i, axis=1)
        self.loss = tf.losses.mean_squared_error(self.y_hat, self.y_i)
        
        self.global_step = tf.Variable(0, trainable=False)
        self.starter_learning_rate = 0.00001
        self.learning_rate = tf.train.exponential_decay(self.starter_learning_rate, self.global_step,
                                                   100000, 0.96, staircase=True)
        self.optimiser = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss) 
        
        self.prediction = tf.round(self.y_hat)
        self.prediction_d = tf.cast(self.prediction, tf.int64)
        
        self.correct = tf.argmax(self.y_i, axis=1)
        self.mistakes = tf.not_equal(self.y_i, self.prediction_d)
        self.accuracy = 1- tf.reduce_mean(tf.cast(self.mistakes, tf.float32))
        
        self.saver = tf.train.Saver()
        
    def train_all(self, x_train, y_train, epoch):
        print('Train with all data, unbatched')
        
        self.train_dict= {self.x_i: x_train,
                          self.y_i: y_train} 
        print('Initial training acc %s' % (sess.run(self.accuracy, feed_dict= self.train_dict)))
        for i in range(epoch):
            sess.run(self.optimiser, feed_dict= self.train_dict)
        print('Final training acc %s' % (sess.run(self.accuracy, feed_dict= self.train_dict)))
        
    def train_full(self, x_train, y_train, epoch, batch_sz, x_valid= None, y_valid= None):

        self.iter_= int(x_train.shape[0]/ batch_sz)
        
        self.train_dict= {self.x_i: x_train,
                          self.y_i: y_train} 
    
        print('Training: Iters: %s. Epoch:  %s' % (self.iter_, epoch))
        print('Initial training acc %s' % (sess.run(self.accuracy, feed_dict= self.train_dict)))
        
        for e in range(epoch):
            e_loss= 0
                
            for i in range(self.iter_):
                iter_dict= {self.x_i: x_train[(i* batch_sz):((i+ 1)* batch_sz)],
                            self.y_i: y_train[(i* batch_sz):((i+ 1)* batch_sz)]}
                
                sess.run(self.optimiser, feed_dict= iter_dict)
                e_loss+= sess.run(self.loss, feed_dict= iter_dict)
                
                #if i %300 == 0:
                #    print('Epoch %s, iter %s, loss %s' % (e, i, sess.run(self.loss, feed_dict= iter_dict)))
                
            print('Epoch %s, loss %s' % (e, sess.run(self.loss, feed_dict= iter_dict)))
        print('Training accuracy %s' % (sess.run(self.accuracy, feed_dict= self.train_dict)))
        
    def predict(self, x_valid):
        valid_dict= {self.x_i: x_valid}  
        predict_1 = (sess.run(self.prediction, feed_dict= valid_dict))
        return predict_1
        
    def test_pointwise(self, x_valid, y_valid):
        self.valid_dict= {self.x_i: x_valid,
                          self.y_i: y_valid} 
        val_accuracy = (sess.run(self.accuracy, feed_dict= self.valid_dict))
        print('Test accuracy: %s' % (val_accuracy))

    def init_disp_var(self, x_array, y_array, iter_= 50):
        sess.run(tf.global_variables_initializer())
        
        self.show_dict= {self.x_i: x_array[:10],
                         self.y_i: y_array[:10]} 
        print('Pre op:', sess.run(self.accuracy, feed_dict= self.show_dict))
        for i in range(iter_):
            sess.run(self.optimiser, feed_dict= self.show_dict)
        print('Post op:', sess.run(self.y_hat, feed_dict= self.show_dict))
        
    def save_model(self, sess, dir_name):
        print('Saving model...')
        if not os.path.exists('./'+ dir_name+ '/'):
            os.mkdir('./'+ dir_name+ '/')
        saver = tf.train.Saver()
        saver.save(sess, './'+ dir_name+ '/model.checkpoint')
        print('Model saved')

    def load_model(self, sess, dir_name):
        print('Loading model...')
        saver = tf.train.Saver()
        saver.restore(sess, './'+ dir_name+ '/model.checkpoint')
        print('Model loaded')
        return sess


In [25]:
print('---Start---')

model1= TF_LogReg()
model1.initilise_model()

with tf.Session() as sess:
    
    print('1. Initialise')
    sess.run(tf.global_variables_initializer())
    model1.load_model(sess, '20170417_LogReg')
    
    model1.test_pointwise(tes_features, tes_ranks_enc)
    
    pred_dictio_eval = evaluate(model1, tes_queries, mean_Xval, std_Xval)
    nDGC_eval_lst = ndcg(pred_dictio_eval)
    ERR_eval_lst = ERR(pred_dictio_eval)
    print('Pre-train nDGC tes: %s, ERR: %s' % (np.nanmean(nDGC_eval_lst), np.nanmean(ERR_eval_lst)))

---Start---
1. Initialise
Loading model...
Model loaded
Test accuracy: 0.527002
Pre-train nDGC tes: 0.268081670047, ERR: 0.109273583201


In [19]:
print('---Start---')

model1= TF_LogReg()
model1.initilise_model()

with tf.Session() as sess:
    
    print('1. Initialise')
    sess.run(tf.global_variables_initializer())
    model1.load_model(sess, '20170419_LogReg')
    
    model1.test_pointwise(tes_features, tes_ranks_enc)
    
    pred_dictio_eval = evaluate(model1, tes_queries, mean_Xval, std_Xval)
    nDGC_eval_lst = ndcg(pred_dictio_eval)
    ERR_eval_lst = ERR(pred_dictio_eval)
    print('Pre-train nDGC tes: %s, ERR: %s' % (np.nanmean(nDGC_eval_lst), np.nanmean(ERR_eval_lst)))
    
    print('2. Train')
    model1.train_full(features, ranks_enc, 1, 30)
    
    print('3. Test')
    model1.test_pointwise(tes_features, tes_ranks_enc)
    
    pred_dictio_eval = evaluate(model1, tes_queries, mean_Xval, std_Xval)
    nDGC_eval_lst = ndcg(pred_dictio_eval)
    ERR_eval_lst = ERR(pred_dictio_eval)
    
    print('Post-train nDGC tes: %s, ERR: %s' % (np.nanmean(nDGC_eval_lst), np.nanmean(ERR_eval_lst)))
    model1.save_model(sess, '20170419_LogReg')
    
print('---End---')

---Start---
1. Initialise
Test accuracy: 0.240575
Pre-train nDGC tes: 0.204874215314, ERR: 0.49609029241
2. Train
Training: Iters: 24113. Epoch:  1
Initial training acc 0.242126
Epoch 0, loss 1.00344
Training accuracy 0.554955
3. Test
Test accuracy: 0.541965
Post-train nDGC tes: 0.252887304128, ERR: 0.113063978213
---End---


In [22]:
print(pd.DataFrame(pred_dictio_eval[58])[:10])

   0  1
0  0  4
1  0  4
2  2  3
3  0  3
4  1  2
5  0  2
6  0  1
7  0  1
8  0  1
9  0  1
